In [57]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import math as m

In [58]:
data = pd.read_csv('/home/stepbruh/Рабочий стол/Практикум/Специалист по Data Science Плюс. Часть 13 из 16 (2022)/Модели и алгоритмы в машинном обучении/04 Обучение нейронной сети/Electrical_Grid_Stability.csv', sep=';')

In [59]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

In [60]:
X =  torch.FloatTensor(data.drop(['stability', 'Unnamed: 0'], axis = 1).values)
y = torch.FloatTensor(data.stability.values)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3, shuffle=True, random_state=42)

In [62]:
n_in_neurons = 12
n_hidden_neurons_1 = 8
n_hidden_neurons_2 = 4
n_out_neurons = 1

In [63]:
model = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_1,n_hidden_neurons_2),
    nn.Tanh(),
    nn.Linear(n_hidden_neurons_2,n_out_neurons),
    nn.Sigmoid()
)

In [64]:
print(model)

Sequential(
  (0): Linear(in_features=12, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): Tanh()
  (4): Linear(in_features=4, out_features=1, bias=True)
  (5): Sigmoid()
)


In [65]:
optimizer = torch.optim.Adam(params=model.parameters(),lr=1e-3)
loss = nn.BCELoss()
num_epochs = 100
batch_length = 500
num_batches = m.ceil((len(X_train))/batch_length)

In [66]:
for epoch in range(num_epochs):

    order = np.random.permutation(len(X_train))
    for batch_idx in range(num_batches):

        start_idx = batch_length * batch_idx
        optimizer.zero_grad()

        batch_idxs = start_idx + batch_length
        X_batch = X_train[start_idx:batch_idxs]
        y_batch = y_train[start_idx:batch_idxs]

        preds = model.forward(X_batch).flatten()
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        optimizer.step()

    if epoch%10==0 or epoch==num_epochs-1:
        model.eval()
        with torch.no_grad():
            test_preds = model.forward(X_test)
            accuracy = (torch.round(test_preds) == y_test).float().mean().item()
        print(f"Epoch {epoch}: Loss = {loss_value.item():.4f}, Accuracy = {accuracy:.4f}")


Epoch 0: Loss = 0.7172, Accuracy = 0.6463
Epoch 10: Loss = 0.6033, Accuracy = 0.6181
Epoch 20: Loss = 0.5394, Accuracy = 0.5755
Epoch 30: Loss = 0.4725, Accuracy = 0.5630
Epoch 40: Loss = 0.4092, Accuracy = 0.5537
Epoch 50: Loss = 0.3672, Accuracy = 0.5484
Epoch 60: Loss = 0.3384, Accuracy = 0.5467
Epoch 70: Loss = 0.3166, Accuracy = 0.5462
Epoch 80: Loss = 0.2960, Accuracy = 0.5453
Epoch 90: Loss = 0.2815, Accuracy = 0.5462
Epoch 99: Loss = 0.2725, Accuracy = 0.5461
